## Install Dependencies and Initial Plan


1. Model Setup (Ollama + DeepSeek)

2. Data Preparation (Knowledge Source)

3. Chunk & Embed Data

4. Build Vector DB

5. Query Pipeline (RAG Flow)

6. Streamlit Setup (UI)

In [27]:
!pip install langchain langchain-community langchain-core



You should consider upgrading via the 'F:\Chatbot\chatbot_env\Scripts\python.exe -m pip install --upgrade pip' command.


## Model Setup (Ollama + DeepSeek)

In [28]:
import ollama

# Function to get response from Ollama DeepSeek 1.5B
def ollama_response(query: str) -> str:
    response = ollama.chat(model="llama3.2:3b", messages=[{"role": "user", "content": query}])
    return response['message']['content']

# Testing the setup with a simple query
print(ollama_response("Hello, how are you?"))


I'm just a language model, so I don't have emotions or feelings like humans do. However, I'm functioning properly and ready to assist you with any questions or tasks you may have! How can I help you today?


In [29]:
from langchain_community.llms import Ollama
from langchain_core.prompts import ChatPromptTemplate

# Instantiate the model
llm = Ollama(model="llama3.2:3b")

# Example prompt setup
prompt = ChatPromptTemplate.from_template("Tell me a joke about AI.")

# Run the model with the prompt
response = llm.invoke(prompt.format())

print(response)


Here's one:

Why did the AI program go to therapy?

Because it was struggling to process its emotions! (get it?)


## Data Preparation (Knowledge Source)

In [30]:
#Create .txt, .pdf, or .md file (example: docs/my_faqs.txt)

#Keep clean, structured knowledge (questions, answers, docs)

import os

def load_documents_from_folder(folder_path: str):
    documents = []

    # Ensure folder exists
    os.makedirs(folder_path, exist_ok=True)

    # Check for .txt files
    txt_files = [f for f in os.listdir(folder_path) if f.endswith(".txt")]

    # If no .txt file exists, create one
    if not txt_files:
        default_path = os.path.join(folder_path, "knowledge.txt")
        with open(default_path, 'w', encoding='utf-8') as f:
            f.write("This is a default knowledge base. Please add more .txt files.")
        txt_files.append("knowledge.txt")

    # Load all .txt files
    for filename in txt_files:
        with open(os.path.join(folder_path, filename), 'r', encoding='utf-8') as file:
            documents.append(file.read())

    return documents

# Example usage
folder_path = "./knowledge_source"
documents = load_documents_from_folder(folder_path)
print(documents[:1])  # print the first document






['Welcome to **Upskill** — your gateway to a future in Software Quality Assurance (SQA)!\n\n**Why Choose Upskill?**\nAt Upskill, we don’t just teach — we transform beginners into QA professionals ready for the global tech industry. Our SQA course is crafted by industry experts to meet real-world needs.\n\n---\n\n**What You’ll Learn in the SQA Course:**\n✅ Fundamentals of Software Testing  \n✅ Test Planning & Case Design  \n✅ Manual & Automated Testing Techniques  \n✅ Tools: Selenium, JIRA, Postman, Git  \n✅ Bug Reporting & Defect Life Cycle  \n✅ Agile & DevOps Essentials  \n✅ Resume Building + Mock Interviews  \n\n---\n\n**Who is this for?**\n- Fresh graduates looking to enter tech  \n- Career switchers from non-IT backgrounds  \n- Professionals seeking QA upskilling  \nNo coding background? No problem! We’ll guide you step-by-step.\n\n---\n\n**100% Online | Live Instructor-Led Classes | Hands-On Projects**\n\n**Real Projects + Internship Opportunity**\nWork on real-world testing proje

## Chunk & Embed Data

In [31]:
#Use LangChain to:

#Load your documents

#Split into smaller chunks

#Embed with HuggingFaceEmbeddings

#Store into Chroma vector DB


from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter

# Use a small, fast model for local embedding
embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Chunking function
def chunk_documents(documents, chunk_size=500, chunk_overlap=50):
    text_splitter = CharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    chunks = []
    for doc in documents:
        chunks.extend(text_splitter.split_text(doc))
    return chunks

# Load knowledge from file (assuming knowledge.txt exists)
def load_knowledge_file():
    file_path = "knowledge_source/knowledge.txt"
    try:
        with open(file_path, "r", encoding="utf-8") as file:
            content = file.read()
            print("✅ knowledge.txt loaded successfully.")
            return content
    except FileNotFoundError:
        print(f"❌ File not found: {file_path}")
        return ""
    except Exception as e:
        print(f"⚠️ Error reading file: {e}")
        return ""

# Load knowledge base
knowledge_text = load_knowledge_file()

# If knowledge is loaded, chunk it
if knowledge_text:
    documents = [knowledge_text]  # Wrap the text into a list to pass to chunk_documents
    chunks = chunk_documents(documents)
    print(f"First chunk: {chunks[:1]}")  # Show the first chunk
else:
    print("⚠️ No knowledge to chunk.")




✅ knowledge.txt loaded successfully.
First chunk: ['Welcome to **Upskill** — your gateway to a future in Software Quality Assurance (SQA)!\n\n**Why Choose Upskill?**\nAt Upskill, we don’t just teach — we transform beginners into QA professionals ready for the global tech industry. Our SQA course is crafted by industry experts to meet real-world needs.\n\n---']


## Build Vector DB



In [32]:
import chromadb
from langchain.vectorstores import Chroma

# Initialize Chroma client
client = chromadb.Client()

# Create or load the collection for storing the vectors
persist_directory = "./chromadb_persist"
if not os.path.exists(persist_directory):
    os.mkdir(persist_directory)

# Create collection in Chroma
collection = client.get_or_create_collection(name="chatbot_collection")


# Create the Chroma vector store
vectorstore = Chroma(persist_directory=persist_directory, embedding_function=embedding)

# Add chunks to Chroma DB
vectorstore.add_texts(chunks)
print("Chunks added to vector DB!")



# working on this to find solution

Chunks added to vector DB!


## Query Pipeline (RAG Flow)



In [33]:
#Accept user question

#Convert to embedding

#Search top-k similar chunks from Chroma

#Concatenate chunks + question

#Send to OllamaLLM via LangChain

#Get DeepSeek response



from langchain.chains import ConversationalRetrievalChain
from langchain.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# Load the Hugging Face model and tokenizer
model_name = "sentence-transformers/all-MiniLM-L6-v2"  # Replace with your desired model
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Use the pipeline to create a conversational model
huggingface_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Initialize HuggingFacePipeline directly without BaseChatModel
llm = HuggingFacePipeline(pipeline=huggingface_pipeline)

# Initialize the retrieval QA system with the Hugging Face model
qa_chain = ConversationalRetrievalChain.from_llm(llm, vectorstore.as_retriever())

# Function to process query using RAG flow
def query_pipeline(query: str) -> str:
    # Initialize an empty chat history
    chat_history = []
    
    # Create a dictionary with the 'question' and 'chat_history' keys
    input_data = {"question": query, "chat_history": chat_history}
    
    # Run the query through the QA chain
    response = qa_chain.run(input_data)
    return response

# Test the query pipeline
print(query_pipeline("why sqa is a great careerpath?"))  # Replace with your question





If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`
Some weights of BertLMHeadModel were not initialized from the model checkpoint at sentence-transformers/all-MiniLM-L6-v2 and are newly initialized: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cpu


Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

---

🎯 **Why SQA is a Great Career Path?**
- High demand globally  
- Great salary growth  
- Opportunities to work in agile teams  
- A critical role in product quality & user satisfaction  

---

📞 **Contact Upskill Today**
- Website: www.upskill.com  
- WhatsApp: +8801XXXXXXXXX  
- Email: info@upskill.com

🧠 Learn QA. Get Certified. Launch Your Career — with Upskill.

---

🎯 **Why SQA is a Great Career Path?**
- High demand globally  
- Great salary growth  
- Opportunities to work in agile teams  
- A critical role in product quality & user satisfaction  

---

📞 **Contact Upskill Today**
- Website: www.upskill.com  
- WhatsApp: +8801XXXXXXXXX  
- Email: info@upskill.com

🧠 Learn QA. Get Certified. Launch Your Career — with Upskill.

---

🎯 **Why SQA is a Great Career Path?**
- High demand globally  
- Great salary growt

In [34]:
from langchain.chains import ConversationalRetrievalChain
from langchain_community.llms import Ollama

# Load the Ollama LLM (llama3.2)
llm = Ollama(model="llama3.2:3b")

# Use your already setup Chroma vectorstore
# Assume variable name is `vectorstore`

# Create the Retrieval Chain
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=vectorstore.as_retriever()
)

#  Query Function
def query_pipeline(query: str, chat_history: list = []) -> str:
    input_data = {"question": query, "chat_history": chat_history}
    response = qa_chain.invoke(input_data)
    return response["answer"]

# Test it
print(query_pipeline("Who is the CEO of Upskill?"))


I don't know who the CEO of Upskill is. The provided context only mentions the mission and benefits of the Upskill program, but does not include any information about the organization's leadership or specific individuals in key roles.


In [35]:
from langchain_community.llms import Ollama
from langchain.chains import ConversationalRetrievalChain

# === Load Ollama Model ===
# Make sure your model (like deepseek:1.5b) is running via `ollama run deepseek`
llm = Ollama(model="deepseek-r1:1.5b")

# === Build Retrieval-Augmented Generation Chain ===
qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=vectorstore.as_retriever()  # Make sure vectorstore is initialized with your docs
)

# === Query Function with Chat History ===
chat_history = []

def query_pipeline(query: str) -> str:
    global chat_history
    response = qa_chain.invoke({
        "question": query,
        "chat_history": chat_history
    })
    # Update chat history for conversational memory
    chat_history.append((query, response["answer"]))
    return response["answer"]


In [36]:
print(query_pipeline("Tell me about Upskill"))



<think>
Okay, so I need to figure out what "Upskill" is based on the given context. Let me read through it again carefully. 

The first line says, "Welcome to Upskill — your gateway to a future in Software Quality Assurance (SQA)!" That immediately tells me that Upskill is an educational resource focused on SQA. It's part of a series, but all seem to lead up to the same main point.

Then it mentions why choose Upskill: they don't just teach; they transform beginners into QA professionals ready for the global tech industry. So, clearly,Upskill provides not only basic training but helps learners transition smoothly to becoming a QA professional in an IT field. They're preparing people for real-world demands in the tech industry.

I also see that it's crafted by industry experts, which suggests it's based on real-world expertise and practical needs. The course is designed to meet the real-world needs of SQA professionals, meaning they've considered what skills are most important for trans

## Streamlit Setup (UI)